# CityBikes

Send a request to CityBikes for the city of your choice. 

In [5]:
import requests
import pandas as pd

# Fetch the list of all networks
response = requests.get("http://api.citybik.es/v2/networks")
data = response.json()

# Check if the request was successful (status code 200)
if response.status_code == 200:
    city_name = input("Enter name of city: ")

    filtered_networks = [network for network in data["networks"] if network["location"]["city"].lower() == city_name.lower()]

    if filtered_networks:
        chosen_network = filtered_networks[0]
        chosen_network_id = chosen_network["id"]

        chosen_network_response = requests.get(f"http://api.citybik.es/v2/networks/{chosen_network_id}")
        chosen_network_data = chosen_network_response.json()

        bike_stations = chosen_network_data.get("network", {}).get("stations", [])

        if bike_stations:
            # Create a DataFrame from the bike_stations list
            df = pd.DataFrame(bike_stations)

            print(f"Bike Stations in {city_name}:\n")
            print(df)
        else:
            print(f"No bike stations found for {city_name}.")
    else:
        print(f"No networks found for {city_name}.")
else:
    print(f"Failed to retrieve network data. Status code: {response.status_code}")


Bike Stations in porto alegre:

    empty_slots                                              extra  \
0             7  {'address': 'Avenida Ipiranga / canteiro centr...   
1             8  {'address': 'Rua Mucio Teixeira calçada do sup...   
2             7  {'address': 'Rua José do Patrocínio oposto ao ...   
3            10  {'address': 'Avenida Praia de Belas frente ao ...   
4            14  {'address': 'R. Cecília Meireles, esquina com ...   
..          ...                                                ...   
90           15  {'address': 'Avenida Bento Gonçalves, oposto a...   
91            6  {'address': 'Rua Jaraguá, oposto ao N 696', 'a...   
92           18  {'address': 'Av. João Walig, próximo da Av. Tú...   
93           14  {'address': 'Avenida Túlio de Rose, E/F ao N 3...   
94           10  {'address': 'Rua Mata Bacelar, na lateral do N...   

    free_bikes                                id   latitude  longitude  \
0            8  350d38f76b6f9c980a3a587cd3e4ed41 -30.

In [6]:
df.info(), print(type(df))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   empty_slots  95 non-null     int64  
 1   extra        95 non-null     object 
 2   free_bikes   95 non-null     int64  
 3   id           95 non-null     object 
 4   latitude     95 non-null     float64
 5   longitude    95 non-null     float64
 6   name         95 non-null     object 
 7   timestamp    95 non-null     object 
dtypes: float64(2), int64(2), object(4)
memory usage: 6.1+ KB
<class 'pandas.core.frame.DataFrame'>


(None, None)

In [7]:
# Notice a column named extra which I dont know it's content.
# Let's explore it.
print(type(df['extra']))


<class 'pandas.core.series.Series'>


Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [8]:
# Create function to retrieve columns of interest
def get_bike_stations_data(city_name):
    filtered_networks = []
    for network in data["networks"]:
        if network["location"]["city"].lower() == city_name.lower():
            filtered_networks.append(network)

    if filtered_networks:
        chosen_network = filtered_networks[0]
        chosen_network_id = chosen_network["id"]

        chosen_network_response = requests.get(f"http://api.citybik.es/v2/networks/{chosen_network_id}")
        chosen_network_data = chosen_network_response.json()

        bike_stations = chosen_network_data["network"]["stations"]

        # Create a list of dictionaries to store station data
        station_data_list = []

        for station in bike_stations:
            latitude = station["latitude"]
            longitude = station["longitude"]
            station_name = station["name"]  # Include station name

            # Check if 'ebikes' is in the 'extra' dictionary
            if 'extra' in station and 'ebikes' in station['extra']:
                total_bikes = station["free_bikes"] + station["empty_slots"] + station['extra']['ebikes']
            else:
                total_bikes = station["free_bikes"] + station["empty_slots"]

            station_data = {
                "Station Name": station_name,
                "Latitude": latitude,
                "Longitude": longitude,
                "Total Bikes": total_bikes
            }

            station_data_list.append(station_data)

        # Create a DataFrame from the list of dictionaries
        df = pd.DataFrame(station_data_list)

        print(f"Bike Stations in {city_name} (DataFrame):\n")
        print(df)

        # Return the DataFrame
        return df
    else:
        print(f"No networks found for {city_name}.")
        # Return an empty DataFrame if no networks are found
        return pd.DataFrame()

# Fetch the list of all networks
response = requests.get("http://api.citybik.es/v2/networks")
data = response.json()

# Take user input for the city name
city_name = city_name

# Call the function to retrieve and display bike station data and save it in the 'city_bike_df' variable
city_bike_df = get_bike_stations_data(city_name)


Bike Stations in porto alegre (DataFrame):

                        Station Name   Latitude  Longitude  Total Bikes
0                    24 - Cascatinha -30.048463 -51.218161           15
1              25 - Estação Nacional -30.044404 -51.223403           23
2               26 - Praça Garibaldi -30.043309 -51.219765           23
3                20 - Praia de Belas -30.047039 -51.227530           15
4                  21 - Praça Itália -30.051020 -51.229523           19
..                               ...        ...        ...          ...
90       75 - Acesso Bento Gonçalves -30.061797 -51.174298           19
91               65 - Praça da Encol -30.035113 -51.186051           15
92  74 - Shopping Iguatemi / Bourbon -30.022538 -51.163716           23
93              73 - Parque Germânia -30.022685 -51.158890           19
94      96 - Mata Bacelar - Mariland -30.020714 -51.193835           15

[95 rows x 4 columns]


In [9]:
# Review and compare make up of the dataframe
city_bike_df.shape, city_bike_df.info(), 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Station Name  95 non-null     object 
 1   Latitude      95 non-null     float64
 2   Longitude     95 non-null     float64
 3   Total Bikes   95 non-null     int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 3.1+ KB


((95, 4), None)

Put your parsed results into a DataFrame.

In [10]:
# Parsed into a dataframe in the function above


In [11]:
# We then view structure of df and city_bike_df
df.info(), city_bike_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   empty_slots  95 non-null     int64  
 1   extra        95 non-null     object 
 2   free_bikes   95 non-null     int64  
 3   id           95 non-null     object 
 4   latitude     95 non-null     float64
 5   longitude    95 non-null     float64
 6   name         95 non-null     object 
 7   timestamp    95 non-null     object 
dtypes: float64(2), int64(2), object(4)
memory usage: 6.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Station Name  95 non-null     object 
 1   Latitude      95 non-null     float64
 2   Longitude     95 non-null     float64
 3   Total Bikes   95 non-null     int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 3.

(None, None)

##### Next step is to merge the two dataframes together

In [12]:

# Concatenate the two DataFrames along the columns axis (axis=1)
merged_city = pd.concat([df, city_bike_df], axis=1)

# Display the merged DataFrame
merged_city.info() # This gives us same number of rows and 15 columns. Making progress



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   empty_slots   95 non-null     int64  
 1   extra         95 non-null     object 
 2   free_bikes    95 non-null     int64  
 3   id            95 non-null     object 
 4   latitude      95 non-null     float64
 5   longitude     95 non-null     float64
 6   name          95 non-null     object 
 7   timestamp     95 non-null     object 
 8   Station Name  95 non-null     object 
 9   Latitude      95 non-null     float64
 10  Longitude     95 non-null     float64
 11  Total Bikes   95 non-null     int64  
dtypes: float64(4), int64(3), object(5)
memory usage: 9.0+ KB


In [13]:
# View merged_city
merged_city.head(2)

,empty_slots,extra,free_bikes,id,latitude,longitude,name,timestamp,Station Name,Latitude,Longitude,Total Bikes
0,7,{'address': 'Avenida Ipiranga / canteiro centr...,8,350d38f76b6f9c980a3a587cd3e4ed41,-30.048463,-51.218161,24 - Cascatinha,2023-09-09T13:05:37.942000Z,24 - Cascatinha,-30.048463,-51.218161,15
1,8,{'address': 'Rua Mucio Teixeira calçada do sup...,15,5c45697c0c6b6a96e39175bcd58e2e7f,-30.044404,-51.223403,25 - Estação Nacional,2023-09-09T13:05:37.944000Z,25 - Estação Nacional,-30.044404,-51.223403,23


In [14]:
# Already put in a dataframe in the function above.
print(type(merged_city))

<class 'pandas.core.frame.DataFrame'>


In [17]:
merged_city.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   empty_slots   95 non-null     int64  
 1   extra         95 non-null     object 
 2   free_bikes    95 non-null     int64  
 3   id            95 non-null     object 
 4   latitude      95 non-null     float64
 5   longitude     95 non-null     float64
 6   name          95 non-null     object 
 7   timestamp     95 non-null     object 
 8   Station Name  95 non-null     object 
 9   Latitude      95 non-null     float64
 10  Longitude     95 non-null     float64
 11  Total Bikes   95 non-null     int64  
dtypes: float64(4), int64(3), object(5)
memory usage: 9.0+ KB


In [15]:
merged_city.to_csv(r'C:\Users\affuy\Documents\Data_Sets\df_2.csv')
